## Importing Nltk library

In [15]:
import nltk

## Importing other required modules

In [16]:
import pandas as pd

In [17]:
fake = pd.read_csv("/kaggle/input/fake-news-detection/Fake.csv")
genuine = pd.read_csv("/kaggle/input/fake-news-detection/True.csv")

### Printing Info

In [18]:
print(fake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None


In [19]:
print(genuine.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None


In [20]:
fake['target']=0
genuine['target']=1

In [21]:
genuine.head()

,title,text,subject,date,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [22]:
fake.head()

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


## Concatenating both datasets

In [23]:
df=pd.concat([fake,genuine],axis=0) 
df=df.reset_index(drop=True)

Dropping unnecessary columns

In [24]:
df=df.drop(['subject','date','title'],axis=1)
df.columns

Index(['text', 'target'], dtype='object')

### Preprocessing data

1. Tokenizing words

In [25]:
from nltk.tokenize import word_tokenize
df['text']=df['text'].apply(word_tokenize)
df

,text,target
0,"[Donald, Trump, just, couldn, t, wish, all, Am...",0
1,"[House, Intelligence, Committee, Chairman, Dev...",0
2,"[On, Friday, ,, it, was, revealed, that, forme...",0
3,"[On, Christmas, day, ,, Donald, Trump, announc...",0
4,"[Pope, Francis, used, his, annual, Christmas, ...",0
...,...,...
44893,"[BRUSSELS, (, Reuters, ), -, NATO, allies, on,...",1
44894,"[LONDON, (, Reuters, ), -, LexisNexis, ,, a, p...",1
44895,"[MINSK, (, Reuters, ), -, In, the, shadow, of,...",1
44896,"[MOSCOW, (, Reuters, ), -, Vatican, Secretary,...",1


2. Stemming Words

In [26]:
from nltk.stem.snowball import SnowballStemmer
ss=SnowballStemmer("english")

In [27]:
def stem_word(text):
    return[ss.stem(word) for word in text]

In [28]:
df['text']=df['text'].apply(stem_word)
df

,text,target
0,"[donald, trump, just, couldn, t, wish, all, am...",0
1,"[hous, intellig, committe, chairman, devin, nu...",0
2,"[on, friday, ,, it, was, reveal, that, former,...",0
3,"[on, christma, day, ,, donald, trump, announc,...",0
4,"[pope, franci, use, his, annual, christma, day...",0
...,...,...
44893,"[brussel, (, reuter, ), -, nato, alli, on, tue...",1
44894,"[london, (, reuter, ), -, lexisnexi, ,, a, pro...",1
44895,"[minsk, (, reuter, ), -, in, the, shadow, of, ...",1
44896,"[moscow, (, reuter, ), -, vatican, secretari, ...",1


### Stopword Removal

In [29]:
from nltk.corpus import stopwords

In [30]:
def rem_stopwords(text):
    dt=[word for word in text if len(word)>2]
    return dt

In [31]:
df['text']=df['text'].apply(rem_stopwords)
df

,text,target
0,"[donald, trump, just, couldn, wish, all, ameri...",0
1,"[hous, intellig, committe, chairman, devin, nu...",0
2,"[friday, was, reveal, that, former, milwauke, ...",0
3,"[christma, day, donald, trump, announc, that, ...",0
4,"[pope, franci, use, his, annual, christma, day...",0
...,...,...
44893,"[brussel, reuter, nato, alli, tuesday, welcom,...",1
44894,"[london, reuter, lexisnexi, provid, legal, reg...",1
44895,"[minsk, reuter, the, shadow, disus, soviet-era...",1
44896,"[moscow, reuter, vatican, secretari, state, ca...",1


### Join text

In [32]:
df['text']=df['text'].apply(' '.join)

## Splitting to Train and test data

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['text'],df['target'])

## Vectorization

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
Iv= TfidfVectorizer(max_df=0.7)

In [36]:
X_train = Iv.fit_transform(X_train)
X_test=Iv.transform(X_test)

In [42]:
print(X_train[:5])

  (0, 17609)	0.10223015353449519
  (0, 23706)	0.07036007135505687
  (0, 55279)	0.03305318719856432
  (0, 67783)	0.060141972617244494
  (0, 48590)	0.17643010311355822
  (0, 87990)	0.034405097670943595
  (0, 21616)	0.06444142109167973
  (0, 60665)	0.05984080027832769
  (0, 7348)	0.03101058078216202
  (0, 68879)	0.038567960459505286
  (0, 23786)	0.07614450744951115
  (0, 73585)	0.05494480321368223
  (0, 69167)	0.06306710145580778
  (0, 48394)	0.08522414497858441
  (0, 80535)	0.05000028495026233
  (0, 60671)	0.14814071117567773
  (0, 89191)	0.10605333435962874
  (0, 60465)	0.06204772712116852
  (0, 57494)	0.14780445789081595
  (0, 41457)	0.1016469128420906
  (0, 55439)	0.04306081266931413
  (0, 60314)	0.06656060540358368
  (0, 49211)	0.0803406078430091
  (0, 74044)	0.11705487552919973
  (0, 60620)	0.05478589405286061
  :	:
  (4, 20281)	0.09675262419433646
  (4, 17508)	0.20393512207228356
  (4, 79717)	0.23541121716426544
  (4, 65785)	0.10281214935074322
  (4, 46616)	0.10809385805945455
  (4

In [43]:
print(X_test)

  (0, 88539)	0.10229270865219552
  (0, 88227)	0.0886943509679203
  (0, 88205)	0.05621770867662579
  (0, 84703)	0.06749808341085985
  (0, 82836)	0.05118598901857564
  (0, 82408)	0.11530532768849165
  (0, 81777)	0.17816024347524853
  (0, 81081)	0.07462299217799338
  (0, 80919)	0.048456385953195985
  (0, 80727)	0.05284438420994256
  (0, 77232)	0.05159254561566746
  (0, 76743)	0.10070632112340103
  (0, 73577)	0.1210391617500072
  (0, 72976)	0.10050102224595818
  (0, 72740)	0.1217096171715551
  (0, 69272)	0.05194360494529613
  (0, 65719)	0.09475393099842618
  (0, 65133)	0.049323176310776554
  (0, 62103)	0.3493495575021455
  (0, 59916)	0.07281487831132716
  (0, 56234)	0.10186123637035341
  (0, 55040)	0.08709269792718209
  (0, 51151)	0.08159173795165187
  (0, 43139)	0.29523991621400936
  (0, 37625)	0.04174858900974653
  :	:
  (11224, 18893)	0.04413240198719456
  (11224, 18415)	0.07198314315521907
  (11224, 18040)	0.04166803214265974
  (11224, 17768)	0.04880423764551953
  (11224, 17583)	0.0517

## Training model with Logistic Regression

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [46]:
m=LogisticRegression(max_iter=100)
m.fit(X_train,y_train)
prediction = m.predict(X_test)
print((accuracy_score(y_test,prediction))*100)

98.6815144766147


In [48]:
prediction

array([1, 1, 0, ..., 0, 0, 0])